# Introdução

Web scraping, também conhecido como extração de dados da web, é um método essencial em diversas áreas, como análise de dados, business intelligence, pesquisa de mercado e comércio eletrônico. Essa técnica envolve a coleta e conversão de dados online, incluindo preços de produtos, artigos de notícias, postagens em mídias sociais, informações de contato e dados financeiros, em formatos úteis.

Envolve o envio de solicitações HTTP a um site para obter HTML ou outros dados estruturados, posteriormente analisados e usados para recuperar as informações precisas que você precisa das páginas da Web. Esses dados podem estar na forma de links, textos, imagens, etc.

Em um mundo cada vez mais orientado por dados, o processo de web scraping combina e organiza informações em escala, permitindo que os tomadores de decisão obtenham conhecimento perspicaz, monitorem tendências e façam escolhas sábias. No entanto, a natureza dinâmica das informações da web, a necessidade de metodologias de raspagem confiáveis e flexíveis e as questões éticas e legais (Dilmegani, 2023) fornecem desafios para a extração de dados da web. Esta investigação inicial estabelecerá as bases para a utilização responsável e bem-sucedida da extração de dados da web.

<b>Web Scraping com Python</b>

Web scraping pode ser realizado com a ajuda de ferramentas ou bibliotecas Python. Neste artigo, vamos nos concentrar em usar bibliotecas Python para obter nosso conteúdo desejado de um site. Usaremos a biblioteca BeautifulSoup para realizar web scraping porque:

Beautiful Soup é uma biblioteca Python projetada para analisar documentos HTML e XML. Ele é utilizado para extrair dados da estrutura HTML de páginas web. O Beautiful Soup pode navegar pelo documento HTML, identificando elementos específicos, como qualquer tag HTML com uma determinada classe ou ID e, em seguida, extraindo as informações desejadas.

<b>O Mercado Livre</b>

Foi escolhido o ramo do e-commerce para aplicar a técnica de Web scraping e, dentro desse ramo, o site do Mercado Livre foi escolhido por ser líder de market share do e-commerce no Brasil e também por ser muito popular entre os consumidores.

## Início

### Importando as bibliotecas

Serão importadas as bibliotecas pandas, requests, datetime e BeautifulSoup

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 
from datetime import datetime

Nosso objetivo é verificar as principais ofertas do dia no site. Então a referência de início será a página principal do Mercado Livre contendo essas informações

In [2]:
URL = "https://www.mercadolivre.com.br/ofertas?container_id=MLB779362-1&page=1"
response = requests.get(URL)
page = response.text

In [3]:
with open('mercadolibre.html','w',encoding = "utf-8") as m:
    m.write(page)

In [4]:
doc = BeautifulSoup(page,'html.parser')

A chave para uma web scraping eficaz é entender a estrutura da página da Web de referência e encontrar a informação de que precisa em suas respectivas tags.

Após identificar no site a tag respectiva correspondente ao nome do produto, essa informação é extraída conforme abaixo

In [5]:
#Lendo os nomes dos produtos em html
def get_items_title(doc):
    title_tags = doc.find_all('p', class_='promotion-item__title')
    titles = []
    for tags in title_tags:
        titles.append(tags.text)
    return titles

O mesmo procedimento é feito para a informação dos preços dos produtos

In [6]:
#Lendo os preços dos produtos em HTML
def get_price(doc):
    price_tags = doc.find_all('div', class_ = 'andes-money-amount-combo__main-container')
    price = []
    for tags in price_tags:
        price.append(tags.text.replace('Â',''))
    return price

Como estamos fazendo uma requisição de informações em um site, é preciso monitorar a resposta do mesmo para que o processo não tenha falhas. Esse processo de requests é monitorado conforme abaixo

In [7]:
#Monitorando o status da solicitação
def get_doc(URL):
    response = requests.get(URL)
    doc = BeautifulSoup(response.text, 'html.parser')
    if response.status_code != 200:
        raise Exception ("Failed to load page {}". format(response))
    return doc

Feito isso, agora é agrupar todas as informações em um dataframe

In [8]:
#Agrupando as informações em um dataframe
def scrape_multi_pages(i):
    URL = 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779362-1&page='
    titles,prices = [],[]
    
    for page in range (1, i+1):
        doc = get_doc(URL + str(page))
        titles.extend(get_items_title(doc))
        prices.extend(get_price(doc))
        
        itens = {'produto': titles, 'preco': prices}
        return pd.DataFrame(itens)

Atribuindo à variável a quantidade de páginas a serem realizadas as extrações, convertendo a coluna datetime e retirando os caracteres especiais aplicando regex

In [9]:
#Convertendo a coluna datetime e retirando caracteres especiais
df_ml = scrape_multi_pages(20)
df_ml['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_ml['preco'] = df_ml['preco'].str.extract(r'(\d+[\.,]?\d*)')

In [10]:
df_ml.head()

,produto,preco,scrapy_datetime
0,Creatina Monohidratada 500g - 100% Pura - Sold...,"99,32",2024-03-04 15:08:10
1,Samsung Galaxy A54 5g 256gb 8gb Ram Preto,1.782,2024-03-04 15:08:10
2,Robô Aspirador Wap Robot W90 3 Modos De Limpez...,349,2024-03-04 15:08:10
3,Fechadura Digital Fr 10 De Sobrepor Intelbras,269,2024-03-04 15:08:10
4,Impressora Multifuncional 3 Em 1 Ecotank L3250...,1.099,2024-03-04 15:08:10
